In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 119 kB in 2s (56.1 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
18 packages can be upgraded. Run 'apt list --upgradable' to see them.
openjdk-8-jdk-headless is already the newest version (8u

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# https://drive.google.com/file/d/1jU38cWxHrKGLg1nWO_wW8031z7d1HpnO/view?usp=drive_link

id='1jU38cWxHrKGLg1nWO_wW8031z7d1HpnO'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('googleplaystore.csv')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('googleplaystore.csv', header=True, inferSchema=True)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Desde los ejercicios de pandas sabemos que hay una linea con Rating 19 (mal cargada), lo cual no es posible, la sacamos

In [ ]:
rdd = df.rdd.filter(lambda x: x.App != "Life Made WI-Fi Touchscreen Photo Frame").cache()

Vemos un poco como se presentan los datos del registro

In [ ]:
rdd.take(5)

[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating=4.5, Reviews='215644', Size='25M

Vemos que hay apps repetidas

In [ ]:
aux = rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)
aux1 = aux.filter(lambda x: x[1] > 1)
aux1.count()

798

En mi criterio las sacamos

In [ ]:
rdd1 = rdd.map(lambda x: (x[0], x)).reduceByKey(lambda x, y: x)
rdd1.take(5)

[('Photo Editor & Candy Camera & Grid & ScrapBook',
  Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up')),
 ('Coloring book moana',
  Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up')),
 ('U Launcher Lite – FREE Live Cool Themes, Hide Apps',
  Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver=

Nos quedamos unicamente con la categoria de la app como key y como valor una tupla de la cantidad de reviews de la app (convertida en entero)
con un numero 1, que nos ayudara luego a calcular la cantidad de apps por categoria.

In [ ]:
rdd2 = rdd1.map(lambda x: (x[1][1], (int(x[1][3]), 1)))
rdd2.take(5)

[('ART_AND_DESIGN', (159, 1)),
 ('ART_AND_DESIGN', (967, 1)),
 ('ART_AND_DESIGN', (87510, 1)),
 ('ART_AND_DESIGN', (215644, 1)),
 ('ART_AND_DESIGN', (967, 1))]

Por categoria, sumamos la cantidad de reviews de cada app, y los 1, para saber la cantidad de apps de esa categoria

In [ ]:
rdd3 = rdd2.reduceByKey(lambda a, b : (a[0] + b[0], a[1] + b[1]))
rdd3.take(5)

[('ART_AND_DESIGN', (1419203, 64)),
 ('AUTO_AND_VEHICLES', (1163666, 85)),
 ('BEAUTY', (396240, 53)),
 ('BOOKS_AND_REFERENCE', (16721314, 222)),
 ('BUSINESS', (9890245, 420))]

Calculamos el promedio de reviews de cada categoria

In [ ]:
rdd4 = rdd3.map(lambda x: (x[0], x[1][0] / x[1][1]))
rdd4.take(5)

[('ART_AND_DESIGN', 22175.046875),
 ('AUTO_AND_VEHICLES', 13690.188235294117),
 ('BEAUTY', 7476.226415094339),
 ('BOOKS_AND_REFERENCE', 75321.23423423423),
 ('BUSINESS', 23548.20238095238)]

Y por ultimo nos quedamos con aquella categoria cuyo promedio de reviews sea mayor

In [ ]:
rdd5 = rdd4.reduce(lambda a, b: a if a[1] > b[1] else b)
rdd5

('SOCIAL', 953672.8075313808)